#### Custom Tool Calling and ReAct Agent

**Approach**: Use agents that can select and use multiple tools:
1. **RAG Tool**: For questions about ingested documents
2. **Web Search Tool**: For real-time information and current events
3. **Custom Tools**: For domain-specific data (e.g., stock prices via APIs)

**Agent Capabilities**:
- **Tool Selection**: Automatically chooses the right tool(s) for each query
- **Multi-Step Reasoning**: Can use multiple tools in sequence
- **Context Awareness**: Understands when to use RAG vs. web search vs. custom tools

This creates a comprehensive system that handles both document-based and real-time queries.

In [1]:
# Install notebook dependencies. 
# Will take a while to download and install numerous dependencies. 
# Wait until it finishes before proceeding
%pip install llama_stack_client==0.3.0 yfinance


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
"""
Multi-Tool Agent: Yahoo Finance + Tavily Search

WARNING: ReActAgent is BROKEN in LlamaStack 0.3.0!
- It does NOT execute tools - just hallucinates observations
- The response_format parameter was removed, breaking the tool loop

SOLUTION: Use regular Agent class which uses OpenAI-compatible function calling
"""

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent  # NOT ReActAgent!
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.event_logger import EventLogger
from typing import cast, Iterator
import yfinance as yf

import os

#### We write a custom function that wraps the Yahoo Finance API and decorate with annotations to inform Llamastack that this should be treated as a tool.

The ReActAgent of Llamastack will check the query and call the appropriate tool

In [4]:
@client_tool
def indian_bank_stock(ticker: str, period: str = "current"):
    """
    Fetch stock price and financial data for Indian banks.
    
    :param ticker: Stock ticker symbol (HDFCBANK.NS, ICICIBANK.NS, or SBIN.NS)
    :param period: Time period ("current", "1mo", "3mo", "6mo", "1y")
    :return: Stock information with price, volume, market cap
    """
    print(f"\n{'='*60}")
    print(f"[STOCK TOOL] 🚀 TOOL EXECUTION STARTED")
    print(f"[STOCK TOOL] ticker={ticker}, period={period}")
    print(f"{'='*60}")

    # We are only interested in our competitor bank stocks
    valid_tickers = ["HDFCBANK.NS", "ICICIBANK.NS", "SBIN.NS"]
    if ticker.upper() not in valid_tickers:
        print(f"[STOCK TOOL] Invalid ticker: {ticker}")
        return {"error": f"Only supports: {', '.join(valid_tickers)}", "ticker": ticker}
    
    try:
        stock = yf.Ticker(ticker)
        
        if period != "current":
            hist = stock.history(period=period)
            if hist.empty:
                return {"error": "No data", "ticker": ticker}
            
            first = hist['Close'].iloc[0]
            last = hist['Close'].iloc[-1]
            change_pct = ((last - first) / first) * 100
                        
            return {
                "ticker": ticker.upper(),
                "period": period,
                "start_price": float(first),
                "end_price": float(last),
                "change_percent": float(change_pct),
                "start_date": hist.index[0].strftime('%Y-%m-%d'),
                "end_date": hist.index[-1].strftime('%Y-%m-%d'),
            }
        
        # Current data
        info = stock.info
        price = info.get('currentPrice') or info.get('regularMarketPrice')
        prev = info.get('previousClose')
        
        print(f"[STOCK TOOL] 📊 Yahoo Finance API Response:")
        print(f"[STOCK TOOL]   - Company: {info.get('longName')}")
        print(f"[STOCK TOOL]   - Current Price: ₹{price}")
        print(f"[STOCK TOOL]   - Previous Close: ₹{prev}")
        print(f"[STOCK TOOL]   - Market Cap: {info.get('marketCap')}")
        print(f"[STOCK TOOL]   - Volume: {info.get('volume')}")
        
        result = {
            "ticker": ticker.upper(),
            "company": info.get('longName'),
            "current_price": float(price) if price else None,
            "previous_close": float(prev) if prev else None,
            "market_cap": info.get('marketCap'),
            "volume": info.get('volume'),
        }
        print(f"[STOCK TOOL] ✅ Returning: {result}")
        print(f"{'='*60}\n")
        return result
        
    except Exception as e:
        print(f"[STOCK TOOL] Error: {e}")
        return {"error": str(e)}

In [5]:
# DEBUG: Test Yahoo Finance API directly (without agent)
# This verifies the API is working correctly
print("Testing Yahoo Finance API directly...")
print()

import yfinance as yf

# Test HDFC Bank
print("=== HDFC Bank (HDFCBANK.NS) ===")
hdfc = yf.Ticker("HDFCBANK.NS")
hdfc_info = hdfc.info
print(f"Company: {hdfc_info.get('longName')}")
print(f"Current Price: ₹{hdfc_info.get('currentPrice') or hdfc_info.get('regularMarketPrice')}")
print(f"Previous Close: ₹{hdfc_info.get('previousClose')}")
print()

# Test SBI
print("=== SBI (SBIN.NS) ===")
sbi = yf.Ticker("SBIN.NS")
sbi_info = sbi.info
print(f"Company: {sbi_info.get('longName')}")
print(f"Current Price: ₹{sbi_info.get('currentPrice') or sbi_info.get('regularMarketPrice')}")
print(f"Previous Close: ₹{sbi_info.get('previousClose')}")


Testing Yahoo Finance API directly...

=== HDFC Bank (HDFCBANK.NS) ===
Company: HDFC Bank Limited
Current Price: ₹949.05
Previous Close: ₹962.2

=== SBI (SBIN.NS) ===
Company: State Bank of India
Current Price: ₹1007.15
Previous Close: ₹1018.9


In [ ]:
# LlamaStack service URL (in-cluster)
LLAMASTACK_URL = "http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321"

# For access from Notebooks external to the cluster, use the route URL instead
# LLAMASTACK_URL = "https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/"

# Get Tavily search API key from environment variable with fallback
# Tavily is a search API that provides web search capabilities
tavily_search_api_key = os.getenv('TAVILY_SEARCH_API_KEY', 'tvly-xxxxxxxx')

# Configure provider data for web search
# provider_data: Configuration for external service providers
# tavily_search_api_key: API key for Tavily search service
provider_data = {"tavily_search_api_key": tavily_search_api_key}

# Reinitialize client with provider data for web search
# provider_data enables the client to use external services like Tavily
client = LlamaStackClient(
    base_url=LLAMASTACK_URL,
    provider_data=provider_data,  # Enables web search
    timeout=300.0  # 5 minute timeout for long operations
)

models = client.models.list()
# Model objects still use .identifier in 0.3.0
model_id = next(m.identifier for m in models if m.model_type == "llm")
print(model_id)

INFO:httpx:HTTP Request: GET https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/models "HTTP/1.1 200 OK"


vllm-inference/granite-3-3-8b-instruct


In [18]:
# =============================================================================
# MULTI-TOOL REACTIVE AGENT
# =============================================================================

# Global agent instance (create once, reuse)
_agent = None
_session_id = None

def reset_agent():
    """Reset the agent to force re-initialization with new settings."""
    global _agent, _session_id
    _agent = None
    _session_id = None
    print("Agent reset. Will be re-initialized on next query.")

# Reset agent to pick up new settings
reset_agent()


def initialize_multi_tool_agent():
    """Initialize the agent with both tools (call once)."""
    global _agent, _session_id
    
    if _agent is not None:
        return _agent, _session_id
    
    # Initialize client
    client = LlamaStackClient(
        base_url=LLAMASTACK_URL,
        provider_data={
            "tavily_search_api_key": tavily_search_api_key,
        }
    )
    
    # IMPORTANT: ReActAgent is BROKEN in 0.3.0 - it doesn't execute tools!
    # Using regular Agent which uses OpenAI-compatible function calling
    
    # Define web_search as a proper function tool with explicit schema
    web_search_tool = {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for real-time information including gold prices, exchange rates, news, and any current data",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    }
    
    _agent = Agent(
        client=client,
        model=model_id,
        instructions="""You have access to tools. ALWAYS use the appropriate tool:
- indian_bank_stock: For HDFC Bank (HDFCBANK.NS), ICICI Bank (ICICIBANK.NS), SBI (SBIN.NS) stock prices
- web_search: For exchange rates, gold prices, news, and any other real-time information

NEVER make up data. ALWAYS call the tool and report the exact values returned.""",
        tools=[indian_bank_stock, {"type": "web_search"}],  # Keep simple format, server handles it
    )
    
    _session_id = _agent.create_session("multi-tool-session")
    
    return _agent, _session_id


Agent reset. Will be re-initialized on next query.


In [20]:
import json

def run_stock_query_with_manual_tool_execution(query: str):
    """
    Execute stock queries with MANUAL tool execution.
    The Agent in 0.3.0 doesn't properly execute client-side tools,
    so we need to handle the tool call loop ourselves.
    """
    print(f"User Query: {query}\n")
    print("📈 Using manual tool execution for stock query...\n")
    
    # Define tool schema for LLM
    tool_schema = {
        "type": "function",
        "function": {
            "name": "indian_bank_stock",
            "description": "Fetch stock price for Indian banks: HDFC Bank (HDFCBANK.NS), ICICI Bank (ICICIBANK.NS), SBI (SBIN.NS)",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Stock ticker (HDFCBANK.NS, ICICIBANK.NS, or SBIN.NS)"
                    },
                    "period": {
                        "type": "string",
                        "description": "Time period: current, 1mo, 3mo, 6mo, 1y"
                    }
                },
                "required": ["ticker"]
            }
        }
    }
    
    # Step 1: Ask LLM what tool to call
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the indian_bank_stock tool to get stock prices."},
        {"role": "user", "content": query}
    ]
    
    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        tools=[tool_schema],
        tool_choice="auto"
    )
    
    assistant_message = response.choices[0].message
    print(f"LLM Response: {assistant_message}")
    
    # Step 2: Check if LLM requested a tool call
    if hasattr(assistant_message, 'tool_calls') and assistant_message.tool_calls:
        for tool_call in assistant_message.tool_calls:
            func_name = tool_call.function.name
            func_args = json.loads(tool_call.function.arguments)
            
            print(f"\n🔧 LLM requested tool: {func_name}")
            print(f"   Arguments: {func_args}")
            
            if func_name == "indian_bank_stock":
                # ACTUALLY EXECUTE THE TOOL using yfinance directly
                ticker = func_args.get("ticker", "HDFCBANK.NS")
                period = func_args.get("period", "current")
                
                print(f"\n🚀 EXECUTING TOOL: Fetching {ticker} from Yahoo Finance...")
                stock = yf.Ticker(ticker)
                info = stock.info
                
                result = {
                    "ticker": ticker,
                    "company": info.get('longName'),
                    "current_price": info.get('currentPrice') or info.get('regularMarketPrice'),
                    "previous_close": info.get('previousClose'),
                    "market_cap": info.get('marketCap'),
                    "volume": info.get('volume'),
                }
                print(f"\n📊 Tool Result: {result}")
                
                # Step 3: Send result back to LLM for final response
                messages.append({
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [tool_call]
                })
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result)
                })
                
                final_response = client.chat.completions.create(
                    model=model_id,
                    messages=messages
                )
                
                print(f"\n✅ Final Answer: {final_response.choices[0].message.content}")
    else:
        # No tool call, just print the response
        print(f"Response: {assistant_message.content}")


def run_multi_tool_query(query: str):
    """
    Run a query with smart routing:
    - Stock queries (HDFC, ICICI, SBI) → Manual tool execution
    - Other queries → Responses API with web_search tool
    """
    print(f"User Query: {query}\n")
    
    # Check if this is a stock query for Indian banks
    stock_keywords = ['hdfc', 'icici', 'sbi', 'state bank', 'bank stock', 'stock price']
    is_stock_query = any(kw in query.lower() for kw in stock_keywords)
    
    if is_stock_query:
        run_stock_query_with_manual_tool_execution(query)
    else:
        print("🌐 Routing to Web Search (Responses API)...\n")
        # Use Responses API directly for web search
        response = client.responses.create(
            model=model_id,
            input=query,
            tools=[{"type": "web_search"}],
            stream=False
        )
        print(f"Response: {response.output_text}")

In [21]:
# SHOULD USE indian_bank_stock custom tool
run_multi_tool_query("What's the current stock price of HDFC Bank?")

User Query: What's the current stock price of HDFC Bank?

User Query: What's the current stock price of HDFC Bank?

📈 Using manual tool execution for stock query...



INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/chat/completions "HTTP/1.1 200 OK"


LLM Response: OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParam(role='assistant', content=None, name=None, tool_calls=[OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCall(type='function', id='chatcmpl-tool-d5a045b7ffb541abbd0cd907848f0ca6', function=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCallFunction(arguments='{"ticker": "HDFCBANK.NS", "period": "current"}', name='indian_bank_stock'), index=None)], refusal=None, annotations=None, audio=None, function_call=None, reasoning_content=None)

🔧 LLM requested tool: indian_bank_stock
   Arguments: {'ticker': 'HDFCBANK.NS', 'period': 'current'}

🚀 EXECUTING TOOL: Fetching HDFCBANK.NS from Yahoo Finance...

📊 Tool Result: {'ticker': 'HDFCBANK.NS', 'company': 'HDFC Bank Limited', 'current_price': 949.05, 'previous_close': 962.2, 'market_cap': 14600695185408, 'volume': 52883418}


INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/chat/completions "HTTP/1.1 200 OK"



✅ Final Answer: The current stock price of HDFC Bank is 949.05 as per the latest data. Please note that stock prices are subject to continuous change as the market is open, so the price may have already updated since this information was retrieved. Always check a reliable financial news source or brokerage site for the most current stock prices.


In [14]:
# Test web_search directly using Responses API (bypassing Agent issues)
print("Testing web_search directly via Responses API...")
print()

response = client.responses.create(
    model=model_id,
    input="What is the current gold price per ounce?",
    tools=[{"type": "web_search"}],
    stream=False
)

print(f"Response type: {type(response)}")
print(f"Output: {response.output_text if hasattr(response, 'output_text') else response}")


Testing web_search directly via Responses API...



INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/responses "HTTP/1.1 200 OK"


Response type: <class 'llama_stack_client.types.response_object.ResponseObject'>
Output: The current gold price per ounce is approximately $4,471.82 USD based on recent sources. Always check a reliable financial news or precious metals site for the most current rate.


In [22]:
# SHOULD USE tavily websearch tool
run_multi_tool_query("What is the current rate for gold per ounce?")

User Query: What is the current rate for gold per ounce?

🌐 Routing to Web Search (Responses API)...



INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/responses "HTTP/1.1 200 OK"


Response: According to the most recent web search data, the current spot price for one ounce of gold is approximately $4,462.32 USD. Please note that this price is utilized for reference and might have changed slightly by the time you view this response. It's always wise to check the latest from a commodity market provider before making financial decisions. Here are five sources with this information:

1. [BullionVault - Live Gold Spot Price Chart](https://www.bullionvault.com/gold-price-chart.do) (Score: 0.99915075)
2. [Markets Insider - Gold Spot Price Chart](https://markets.businessinsider.com/commodities/gold-price) (Score: 0.99863297)
3. [Gold Price](https://goldprice.org/gold-price.html) (Score: 0.998487)
4. [Live Gold Price](https://goldavenue.com/en/gold-price/usd/1-oz?srsltid=AfmBOorYRQqkrhiD2yBn1UV5egdCbM8KcGLC1JICrR02pSefO6DRURKQ) (Score: 0.99827254)
5. [JM Bullion - Live Gold Spot Price Charts](https://www.jmbullion.com/charts/gold-price/) (Score: 0.99776566) 

Remember tha